## Install Module

In [5]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())


2.1.0+cu121
12.1
True


In [18]:
!pip uninstall -y torch torchvision torchaudio torchdata dgl

Found existing installation: torch 2.1.0+cu121
Uninstalling torch-2.1.0+cu121:
  Successfully uninstalled torch-2.1.0+cu121
Found existing installation: torchvision 0.16.0+cu121
Uninstalling torchvision-0.16.0+cu121:
  Successfully uninstalled torchvision-0.16.0+cu121
Found existing installation: torchaudio 2.1.0+cu121
Uninstalling torchaudio-2.1.0+cu121:
  Successfully uninstalled torchaudio-2.1.0+cu121
Found existing installation: torchdata 0.7.0
Uninstalling torchdata-0.7.0:
  Successfully uninstalled torchdata-0.7.0
Found existing installation: dgl 2.1.0+cu121
Uninstalling dgl-2.1.0+cu121:
  Successfully uninstalled dgl-2.1.0+cu121


In [1]:
# PyTorch 2.1.0 + CUDA 12.1
!pip install torch==2.1.0+cu121 torchvision==0.16.0+cu121 torchaudio==2.1.0+cu121 --index-url https://download.pytorch.org/whl/cu121

# TorchData 맞춤
!pip install torchdata==0.7.0

# DGL with GraphBolt (CUDA 12.1)
!pip install dgl -f https://data.dgl.ai/wheels/cu121/repo.html

Looking in indexes: https://download.pytorch.org/whl/cu121
Looking in links: https://data.dgl.ai/wheels/cu121/repo.html
  Using cached https://data.dgl.ai/wheels/cu121/dgl-2.1.0%2Bcu121-cp310-cp310-manylinux1_x86_64.whl (467.5 MB)


In [2]:
# 설치 확인 

import torch, torchvision, torchaudio, torchdata, dgl
import dgl.graphbolt as gb

print("Torch:", torch.__version__)
print("TorchData:", torchdata.__version__)
print("Torchvision:", torchvision.__version__)
print("Torchaudio:", torchaudio.__version__)
print("DGL:", dgl.__version__)
print("CUDA:", torch.version.cuda)
print("GPU available:", torch.cuda.is_available())


Torch: 2.1.0+cu121
TorchData: 0.7.0
Torchvision: 0.16.0+cu121
Torchaudio: 2.1.0+cu121
DGL: 2.1.0+cu121
CUDA: 12.1
GPU available: True


In [3]:
# warning message 끄기
!pip install pickleshare --root-user-action=ignore

In [4]:
!pip install tqdm

## Model Training

In [1]:
import torch, torchvision, torchaudio, torchdata, dgl
import dgl.graphbolt as gb

In [2]:
%cd /root/HGMAE

/root/HGMAE


In [3]:
import torch
data = torch.load("data/mydata.pkl")

import torch
data = torch.load("data/mydata.pkl")
print("keys in mydata.pkl:", data.keys())


keys in mydata.pkl: dict_keys(['feats', 'mps', 'nei_index', 'pos'])


In [4]:
import torch
data = torch.load("data/mydata.pkl")

for k in data["feats"]:
    print(k, data["feats"][k].shape)

# paper torch.Size([5000, 832])
# author torch.Size([33607, 768])
# concept torch.Size([6612, 768])

paper torch.Size([5000, 832])
author torch.Size([33607, 768])
concept torch.Size([6612, 768])


In [6]:
for name, edge_index in data["pos"].items():
    print(name, edge_index.shape)

ap torch.Size([2, 40758])
pc torch.Size([2, 82238])
pp torch.Size([2, 8186])
aa torch.Size([2, 984700])


In [7]:
for name, mat in data["mps"].items():
    print(name, mat.shape)


PAP torch.Size([33607, 33607])
PCP torch.Size([5000, 5000])


In [7]:
#!python main.py --dataset mydata --task contrastive --use_cfg

In [1]:
%cd /root/HGMAE
!python main.py --task contrastive --dataset mydata --lr 0.001 --epochs 20

/root/HGMAE
Namespace(seeds=[0], num_heads=4, num_out_heads=1, num_layers=2, residual=False, feat_drop=0.2, attn_drop=0.1, norm=None, lr=0.001, negative_slope=0.2, activation='prelu', feat_mask_rate='0.3,0.05,0.7', replace_rate=0.0, leave_unchanged=0.0, encoder='han', decoder='han', loss_fn='mse', alpha_l=2, optimizer='adam', scheduler=False, scheduler_gamma=0.99, dataset='mydata', ratio=[20, 40, 60], gpu=0, seed=0, hidden_dim=512, mae_epochs=50, eva_lr=0.05, eva_wd=0.0, patience=10, l2_coef=0.0005, use_mp2vec_feat_pred=False, mps_lr=0.005, mps_embedding_dim=64, mps_walk_length=5, mps_context_size=3, mps_walks_per_node=3, mps_num_negative_samples=1, mps_batch_size=128, mps_epoch=20, mp2vec_feat_pred_loss_weight=0.1, mp2vec_feat_alpha_l=2, mp2vec_feat_drop=0.2, use_cfg=False, use_mp_edge_recon=False, mp_edge_recon_loss_weight=1, mp_edge_mask_rate='0.5,0.005,0.8', mp_edge_alpha_l=2, task='contrastive', contrastive_weight=1.0, mae_weight=1.0, tau=0.5, hard_neg_weight=0.0, epochs=20, batch

### 학습 잘 돌아갔는지 확인 

In [4]:
%cd /root/HGMAE
import torch
import pickle

# 파일 로드
with open("pairs_debug.pkl", "rb") as f:
    pairs = pickle.load(f)

print("Top-level keys:", pairs.keys())
print("Subkeys (pos):", pairs["pos"].keys())
print("Subkeys (neg):", pairs["neg"].keys())

def check_edges(edges, name, num_nodes=None):
    print(f"\n=== {name.upper()} ===")
    if edges is None or len(edges) == 0:
        print("⚠️ 없음")
        return None
    
    if isinstance(edges, torch.Tensor):
        e = edges.clone().cpu()
        if e.ndim == 2 and e.shape[0] == 2:
            e = e.t()
    else:
        e = torch.as_tensor(edges, dtype=torch.long)
        if e.ndim == 2 and e.shape[0] == 2:
            e = e.t()
    
    print(f"총 개수: {e.shape[0]}")
    
    uniq = torch.unique(e, dim=0)
    print(f"중복 제거 후 개수: {uniq.shape[0]} (중복 {e.shape[0] - uniq.shape[0]})")
    
    if num_nodes:
        invalid = ((e < 0) | (e >= num_nodes)).any(dim=1).sum().item()
        print(f"유효하지 않은 노드 index 개수: {invalid}")
    
    return e

# 예시: paper 타입만 확인
pos_paper = pairs["pos"]["train"].get("paper")
neg_paper = pairs["neg"]["train"].get("paper")

pos_e = check_edges(pos_paper, "pos_train[paper]", num_nodes=5000)
neg_e = check_edges(neg_paper, "neg_train[paper]", num_nodes=5000)

# pos-neg 교집합
if pos_e is not None and neg_e is not None:
    inter = set(map(tuple, pos_e.tolist())) & set(map(tuple, neg_e.tolist()))
    print(f"\nPos-Neg 교집합 개수: {len(inter)}")
    if len(inter) > 0:
        print("예시 교집합:", list(inter)[:10])


/root/HGMAE
Top-level keys: dict_keys(['pos', 'neg'])
Subkeys (pos): dict_keys(['train', 'val'])
Subkeys (neg): dict_keys(['train', 'val'])

=== POS_TRAIN[PAPER] ===
총 개수: 6823
중복 제거 후 개수: 6823 (중복 0)
유효하지 않은 노드 index 개수: 0

=== NEG_TRAIN[PAPER] ===
총 개수: 6822
중복 제거 후 개수: 6822 (중복 0)
유효하지 않은 노드 index 개수: 0

Pos-Neg 교집합 개수: 0


In [12]:
%cd /root/HGMAE
import torch

data = torch.load("data/mydata.pkl", map_location="cpu")  # GPU → CPU 로드
print("Keys in mydata.pkl:", data.keys())

if "pos" in data:
    pos = data["pos"]
    print("Type of pos:", type(pos))
    if isinstance(pos, dict):
        for k, v in pos.items():
            print(f"{k}: shape={getattr(v, 'shape', None)} type={type(v)}")
    else:
        print("pos content:", pos)



/root/HGMAE
Keys in mydata.pkl: dict_keys(['feats', 'mps', 'nei_index', 'pos'])
Type of pos: <class 'dict'>
ap: shape=torch.Size([2, 40758]) type=<class 'torch.Tensor'>
pc: shape=torch.Size([2, 82238]) type=<class 'torch.Tensor'>
pp: shape=torch.Size([2, 8186]) type=<class 'torch.Tensor'>
aa: shape=torch.Size([2, 984700]) type=<class 'torch.Tensor'>


## Tuning & Evaluation

In [6]:
!grep -r "for epoch in" .

./data/.ipynb_checkpoints/HGMAE_2ndtrial-checkpoint.ipynb:      "./data/.ipynb_checkpoints/HGMAE_2ndtrial-checkpoint.ipynb:      \"./main.py:    for epoch in tqdm(range(args.mae_epochs), desc=\\\"Training\\\", unit=\\\"epoch\\\"):\\n\",\n",
./data/.ipynb_checkpoints/HGMAE_2ndtrial-checkpoint.ipynb:      "./data/.ipynb_checkpoints/HGMAE_2ndtrial-checkpoint.ipynb:      \"./.ipynb_checkpoints/main-checkpoint.py:    for epoch in tqdm(range(args.mae_epochs), desc=\\\"Training\\\", unit=\\\"epoch\\\"):\\n\"\n",
./data/.ipynb_checkpoints/HGMAE_2ndtrial-checkpoint.ipynb:      "./data/.ipynb_checkpoints/HGMAE_2ndtrial-checkpoint.ipynb:    \"!grep -r \\\"for epoch in\\\" .\"\n",
./data/.ipynb_checkpoints/HGMAE_2ndtrial-checkpoint.ipynb:      "./data/HGMAE_2ndtrial.ipynb:      \"./main.py:    for epoch in tqdm(range(args.mae_epochs), desc=\\\"Training\\\", unit=\\\"epoch\\\"):\\n\",\n",
./data/.ipynb_checkpoints/HGMAE_2ndtrial-checkpoint.ipynb:      "./data/HGMAE_2ndtrial.ipynb:      \"./.ipynb_c

In [20]:
!python main.py --task contrastive --dataset mydata --lr 0.001 --epochs 80


Namespace(seeds=[0], num_heads=4, num_out_heads=1, num_layers=2, residual=False, feat_drop=0.2, attn_drop=0.1, norm=None, lr=0.001, negative_slope=0.2, activation='prelu', feat_mask_rate='0.3,0.05,0.7', replace_rate=0.0, leave_unchanged=0.0, encoder='han', decoder='han', loss_fn='mse', alpha_l=2, optimizer='adam', scheduler=False, scheduler_gamma=0.99, dataset='mydata', ratio=[20, 40, 60], gpu=0, seed=0, hidden_dim=512, mae_epochs=50, eva_lr=0.05, eva_wd=0.0, patience=10, l2_coef=0.0005, use_mp2vec_feat_pred=False, mps_lr=0.005, mps_embedding_dim=64, mps_walk_length=5, mps_context_size=3, mps_walks_per_node=3, mps_num_negative_samples=1, mps_batch_size=128, mps_epoch=20, mp2vec_feat_pred_loss_weight=0.1, mp2vec_feat_alpha_l=2, mp2vec_feat_drop=0.2, use_cfg=False, use_mp_edge_recon=False, mp_edge_recon_loss_weight=1, mp_edge_mask_rate='0.5,0.005,0.8', mp_edge_alpha_l=2, task='contrastive', contrastive_weight=1.0, mae_weight=1.0, tau=0.5, hard_neg_weight=0.0, epochs=80, batch_size=128, t

In [8]:
import subprocess
import itertools
import os
import pandas as pd

def sweep_hyperparams(lr_list, epoch_list, dataset="mydata", task="contrastive", seeds=[0]):
    """
    main.py 여러 번 실행 → stdout에서 losfs 값 추출 → 표(DataFrame) 반환
    """
    results = []

    for lr, ep, seed in itertools.product(lr_list, epoch_list, seeds):
        tag = f"lr{lr}_ep{ep}_s{seed}"
        save_dir = os.path.join("runs", tag)
        os.makedirs(save_dir, exist_ok=True)

        cmd = [
            "python", "main.py",
            "--task", task,
            "--dataset", dataset,
            "--lr", str(lr),
            "--epochs", str(ep),
            "--seed", str(seed),
            "--save_dir", save_dir,
            "--exp_name", tag
        ]
        print(">> 실행:", " ".join(cmd))

        # stdout 캡처
        proc = subprocess.run(cmd, capture_output=True, text=True)

        # stdout 라인별 분리
        stdout_lines = proc.stdout.strip().split("\n")

        # "loss" 키워드가 포함된 줄 찾기
        loss_lines = [line for line in stdout_lines if "loss" in line.lower()]
        if loss_lines:
            final_loss_line = loss_lines[-1]  # 마지막 loss 줄
        else:
            final_loss_line = "Not found"

        # 결과 저장
        results.append({
            "lr": lr,
            "epochs": ep,
            "seed": seed,
            "save_dir": save_dir,
            "final_loss": final_loss_line
        })

    # DataFrame으로 정리
    df = pd.DataFrame(results)
    return df


In [43]:
lr_list = [1e-3, 5e-4, 1e-4]
epoch_list = [25,50]

df_results = sweep_hyperparams(lr_list, epoch_list, seeds=[0, 1])
print(df_results)


>> 실행: python main.py --task contrastive --dataset mydata --lr 0.001 --epochs 25 --seed 0 --save_dir runs/lr0.001_ep25_s0 --exp_name lr0.001_ep25_s0
>> 실행: python main.py --task contrastive --dataset mydata --lr 0.001 --epochs 25 --seed 1 --save_dir runs/lr0.001_ep25_s1 --exp_name lr0.001_ep25_s1
>> 실행: python main.py --task contrastive --dataset mydata --lr 0.001 --epochs 50 --seed 0 --save_dir runs/lr0.001_ep50_s0 --exp_name lr0.001_ep50_s0
>> 실행: python main.py --task contrastive --dataset mydata --lr 0.001 --epochs 50 --seed 1 --save_dir runs/lr0.001_ep50_s1 --exp_name lr0.001_ep50_s1
>> 실행: python main.py --task contrastive --dataset mydata --lr 0.0005 --epochs 25 --seed 0 --save_dir runs/lr0.0005_ep25_s0 --exp_name lr0.0005_ep25_s0
>> 실행: python main.py --task contrastive --dataset mydata --lr 0.0005 --epochs 25 --seed 1 --save_dir runs/lr0.0005_ep25_s1 --exp_name lr0.0005_ep25_s1
>> 실행: python main.py --task contrastive --dataset mydata --lr 0.0005 --epochs 50 --seed 0 --save_di

## validation

In [2]:
!python main.py --task contrastive --dataset mydata --lr 0.0005 --epochs 50

python: can't open file '/root/HGMAE/data/main.py': [Errno 2] No such file or directory


In [50]:
lr_list = [1e-3, 5e-4, 1e-4]
epoch_list = [50]

df_results = sweep_hyperparams(lr_list, epoch_list, seeds=[0, 1])
print(df_results)


>> 실행: python main.py --task contrastive --dataset mydata --lr 0.001 --epochs 50 --seed 0 --save_dir runs/lr0.001_ep50_s0 --exp_name lr0.001_ep50_s0
>> 실행: python main.py --task contrastive --dataset mydata --lr 0.001 --epochs 50 --seed 1 --save_dir runs/lr0.001_ep50_s1 --exp_name lr0.001_ep50_s1
>> 실행: python main.py --task contrastive --dataset mydata --lr 0.0005 --epochs 50 --seed 0 --save_dir runs/lr0.0005_ep50_s0 --exp_name lr0.0005_ep50_s0
>> 실행: python main.py --task contrastive --dataset mydata --lr 0.0005 --epochs 50 --seed 1 --save_dir runs/lr0.0005_ep50_s1 --exp_name lr0.0005_ep50_s1
>> 실행: python main.py --task contrastive --dataset mydata --lr 0.0001 --epochs 50 --seed 0 --save_dir runs/lr0.0001_ep50_s0 --exp_name lr0.0001_ep50_s0
>> 실행: python main.py --task contrastive --dataset mydata --lr 0.0001 --epochs 50 --seed 1 --save_dir runs/lr0.0001_ep50_s1 --exp_name lr0.0001_ep50_s1
       lr  epochs  seed               save_dir  \
0  0.0010      50     0   runs/lr0.001_ep50_